In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import pandas as pd
from tkinter.font import Font

# Load the saved model and scaler
model = joblib.load(r'D:\sudee\Academics\house priceing\House_prediction.pkl')
scaler = joblib.load('scaler.pkl')

# EXACT feature order from your dataset
feature_order = [
    'area',
    'mainroad',
    'guestroom',
    'basement',
    'hotwaterheating',
    'airconditioning',
    'prefarea',
    'furnishingstatus_semi_furnished',
    'furnishingstatus_unfurnished',
    'bathrooms_2',
    'bathrooms_3',
    'bathrooms_4',
    'stories_2',
    'stories_3',
    'stories_4',
    'parking_1',
    'parking_2',
    'parking_3',
    'bedrooms_2',
    'bedrooms_3',
    'bedrooms_4',
    'bedrooms_5',
    'bedrooms_6'
]

class GradientFrame(tk.Canvas):
    """A gradient frame which uses a canvas to draw the background"""
    def __init__(self, parent, color1="#3498db", color2="#2c3e50", **kwargs):
        tk.Canvas.__init__(self, parent, **kwargs)
        self._color1 = color1
        self._color2 = color2
        self.bind("<Configure>", self._draw_gradient)
        self._draw_gradient()

    def _draw_gradient(self, event=None):
        """Draw the gradient"""
        self.delete("gradient")
        width = self.winfo_width()
        height = self.winfo_height()
        
        # Get RGB components for each color
        r1, g1, b1 = self._hex_to_rgb(self._color1)
        r2, g2, b2 = self._hex_to_rgb(self._color2)
        
        for i in range(height):
            # Calculate interpolated color
            r = int(r1 * (height - i) / height + r2 * i / height)
            g = int(g1 * (height - i) / height + g2 * i / height)
            b = int(b1 * (height - i) / height + b2 * i / height)
            color = f"#{r:02x}{g:02x}{b:02x}"
            self.create_line(0, i, width, i, tags=("gradient",), fill=color)
        self.lower("gradient")
    
    def _hex_to_rgb(self, hex_color):
        """Convert hex color to RGB tuple"""
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def predict_price():
    try:
        # Create dictionary to store all feature values
        features = {
            'area': float(entries['area'].get())
        }
        
        # Binary features (True/False)
        binary_features = [
            'mainroad', 'guestroom', 'basement',
            'hotwaterheating', 'airconditioning', 'prefarea'
        ]
        for feat in binary_features:
            features[feat] = cat_vars[feat].get() == 'Yes'
        
        # Furnishing status
        furnish_status = cat_vars['furnishingstatus'].get()
        features['furnishingstatus_semi_furnished'] = furnish_status == 'Semi-Furnished'
        features['furnishingstatus_unfurnished'] = furnish_status == 'Unfurnished'
        
        # Bathrooms encoding
        bathrooms = int(entries['bathrooms'].get())
        features['bathrooms_2'] = bathrooms == 2
        features['bathrooms_3'] = bathrooms == 3
        features['bathrooms_4'] = bathrooms == 4
        
        # Stories encoding
        stories = int(entries['stories'].get())
        features['stories_2'] = stories == 2
        features['stories_3'] = stories == 3
        features['stories_4'] = stories == 4
        
        # Parking encoding
        parking = int(entries['parking'].get())
        features['parking_1'] = parking == 1
        features['parking_2'] = parking == 2
        features['parking_3'] = parking == 3
        
        # Bedrooms encoding
        bedrooms = int(entries['bedrooms'].get())
        features['bedrooms_2'] = bedrooms == 2
        features['bedrooms_3'] = bedrooms == 3
        features['bedrooms_4'] = bedrooms == 4
        features['bedrooms_5'] = bedrooms == 5
        features['bedrooms_6'] = bedrooms == 6
        
        # Create DataFrame with EXACT feature order
        input_data = [features[col] for col in feature_order]
        input_df = pd.DataFrame([input_data], columns=feature_order)
        
        # Scale the numerical features (only area in this case)
        input_scaled = scaler.transform(input_df)
        
        # Make prediction
        predicted_price = model.predict(input_scaled)[0]
        
        # Display formatted result with animation
        result_label.config(text="", foreground="#2ecc71")
        root.update()
        display_result_animation(predicted_price)
        
    except Exception as e:
        messagebox.showerror("Input Error", f"Please check your inputs: {str(e)}")

def display_result_animation(final_price):
    current = 0
    step = final_price / 20
    while current < final_price:
        current += step
        if current > final_price:
            current = final_price
        result_label.config(text=f"₹{current:,.2f}")
        root.update()
        root.after(20)

def create_linked_slider_entry(parent, min_val, max_val, default_val, is_integer=True):
    """Create a slider and entry pair that stay in sync"""
    frame = ttk.Frame(parent, style='TFrame')
    
    # Create slider
    slider_var = tk.DoubleVar(value=default_val)
    slider = ttk.Scale(frame, from_=min_val, to=max_val, variable=slider_var,
                      orient='horizontal', style='Horizontal.TScale')
    slider.pack(side='left', fill='x', expand=True, padx=5)
    
    # Create entry
    entry_var = tk.StringVar(value=str(default_val))
    entry = ttk.Entry(frame, textvariable=entry_var, width=8, font=('Helvetica', 10))
    entry.pack(side='left', padx=5)
    
    # Function to update entry from slider
    def update_entry_from_slider(*args):
        val = slider_var.get()
        if is_integer:
            val = int(round(val))
        entry_var.set(str(val))
    
    # Function to update slider from entry
    def update_slider_from_entry(*args):
        try:
            val = float(entry_var.get())
            if val < min_val:
                val = min_val
                entry_var.set(str(val))
            elif val > max_val:
                val = max_val
                entry_var.set(str(val))
            slider_var.set(val)
        except ValueError:
            # If entry is invalid, revert to previous value
            entry_var.set(str(slider_var.get()))
    
    # Set up trace for slider changes
    slider_var.trace_add('write', update_entry_from_slider)
    
    # Set up binding for entry changes
    entry.bind('<FocusOut>', update_slider_from_entry)
    entry.bind('<Return>', update_slider_from_entry)
    
    return frame, slider_var, entry_var

# Create the GUI
root = tk.Tk()
root.title("MULYAMAAN")
root.geometry("1000x800")
root.resizable(True, True)

# Main gradient background
main_gradient = GradientFrame(root, color1="#f5f7fa", color2="#c3cfe2")
main_gradient.pack(fill="both", expand=True)

# Custom styling
style = ttk.Style()
style.theme_use('clam')

# Configure styles
style.configure('TFrame', background="#f5f7fa")
style.configure('TLabel', background="#f5f7fa", font=('Helvetica', 10))
style.configure('TButton', font=('Helvetica', 12, 'bold'), padding=10, 
                background="#3498db", foreground="white")
style.configure('TNotebook', background="#f5f7fa", borderwidth=0)
style.configure('TNotebook.Tab', font=('Helvetica', 10, 'bold'), padding=[10, 5],
                background="#e0e5ec", foreground="#2c3e50")
style.map('TNotebook.Tab', 
          background=[('selected', '#3498db'), ('active', '#2980b9')],
          foreground=[('selected', 'white'), ('active', 'white')])
style.configure('TRadiobutton', background="#f5f7fa", font=('Helvetica', 10))
style.configure('Horizontal.TScale', background="#f5f7fa", troughcolor="#dfe6e9")

# Main container with slight shadow effect
main_container = ttk.Frame(main_gradient, style='TFrame', 
                          relief='raised', borderwidth=1)
main_container.place(relx=0.5, rely=0.5, anchor='center', 
                    width=950, height=700)

# Title frame with gradient
title_frame = GradientFrame(main_container, color1="#3498db", color2="#2c3e50",
                          width=800, height=100)
title_frame.pack(pady=(0, 20))

# Main title
title_font = Font(family="Helvetica", size=24, weight="bold", slant="italic")
title_label = tk.Label(title_frame, 
                      text="MULYAMAAN", 
                      font=title_font, 
                      foreground="white",
                      bg="#3498db")
title_label.place(relx=0.5, rely=0.4, anchor='center')

# Subtitle
subtitle_font = Font(family="Helvetica", size=12)
subtitle_label = tk.Label(title_frame, 
                         text="Enter property details to estimate the market value\nअगर घर हो लेना तो मूल्यमान पर प्राइस चेक करके लेना", 
                         font=subtitle_font, 
                         foreground="#ecf0f1",
                         bg="#3498db",
                         justify='center')
subtitle_label.place(relx=0.5, rely=0.75, anchor='center')

# Create input notebook with modern styling
notebook = ttk.Notebook(main_container, style='TNotebook')
notebook.pack(pady=10, padx=20, fill='both', expand=True)

# Tab for basic features
basic_frame = ttk.Frame(notebook, style='TFrame')
notebook.add(basic_frame, text="🏠 Basic Features")

# Tab for amenities
amenities_frame = ttk.Frame(notebook, style='TFrame')
notebook.add(amenities_frame, text="✨ Amenities")

entries = {}

# Basic Features Tab Content
basic_container = ttk.Frame(basic_frame, style='TFrame')
basic_container.pack(pady=15, padx=20, fill='both', expand=True)

input_fields = [
    ("Area (sq.ft):", "area", "Total built-up area (500-10000)", 500, 10000, 1000, False),
    ("Bedrooms:", "bedrooms", "Number of bedrooms (1-6)", 1, 6, 2, True),
    ("Bathrooms:", "bathrooms", "Number of bathrooms (1-4)", 1, 4, 1, True),
    ("Stories:", "stories", "Number of floors (1-4)", 1, 4, 1, True),
    ("Parking Spaces:", "parking", "Covered parking spaces (0-3)", 0, 3, 0, True)
]

for label_text, field_name, hint_text, min_val, max_val, default_val, is_integer in input_fields:
    frame = ttk.Frame(basic_container, style='TFrame')
    frame.pack(pady=10, fill='x')
    
    # Label with modern styling
    ttk.Label(frame, text=label_text, width=20, anchor='w', 
              font=('Helvetica', 11, 'bold')).pack(side='left')
    
    # Create linked slider and entry
    slider_frame, slider_var, entry_var = create_linked_slider_entry(
        frame, min_val, max_val, default_val, is_integer
    )
    slider_frame.pack(side='left', fill='x', expand=True)
    
    # Store the entry widget for later access
    entries[field_name] = entry_var
    
    # Hint text
    ttk.Label(frame, text=hint_text, 
              font=('Helvetica', 9), foreground="#7f8c8d").pack(side='left')

# Amenities Tab Content
amenities_container = ttk.Frame(amenities_frame, style='TFrame')
amenities_container.pack(pady=15, padx=20, fill='both', expand=True)

cat_vars = {}

amenity_features = [
    ("Main Road Access", "mainroad"),
    ("Guest Room", "guestroom"),
    ("Basement", "basement"),
    ("Hot Water Heating", "hotwaterheating"),
    ("Air Conditioning", "airconditioning"),
    ("Preferred Area", "prefarea")
]

for label_text, cat_feat in amenity_features:
    frame = ttk.Frame(amenities_container, style='TFrame')
    frame.pack(pady=10, fill='x')
    
    ttk.Label(frame, text=f"{label_text}:", width=20, anchor='w', 
              font=('Helvetica', 11, 'bold')).pack(side='left')
    
    var = tk.StringVar(value="No")
    btn_frame = ttk.Frame(frame, style='TFrame')
    btn_frame.pack(side='left', padx=5)
    
    yes_btn = ttk.Radiobutton(btn_frame, text="Yes", value="Yes", variable=var, 
                             style='TRadiobutton')
    yes_btn.pack(side='left', padx=5)
    
    no_btn = ttk.Radiobutton(btn_frame, text="No", value="No", variable=var, 
                            style='TRadiobutton')
    no_btn.pack(side='left', padx=5)
    
    cat_vars[cat_feat] = var

# Furnishing status
frame = ttk.Frame(amenities_container, style='TFrame')
frame.pack(pady=15, fill='x')
ttk.Label(frame, text="Furnishing Status:", width=20, anchor='w', 
          font=('Helvetica', 11, 'bold')).pack(side='left')

furnish_var = tk.StringVar(value="Furnished")
furnish_frame = ttk.Frame(frame, style='TFrame')
furnish_frame.pack(side='left', padx=5)

options = ["Furnished", "Semi-Furnished", "Unfurnished"]
for option in options:
    rb = ttk.Radiobutton(furnish_frame, text=option, value=option, 
                         variable=furnish_var, style='TRadiobutton')
    rb.pack(side='left', padx=5)

cat_vars['furnishingstatus'] = furnish_var

# Prediction button with modern style
button_frame = ttk.Frame(main_container, style='TFrame')
button_frame.pack(pady=20)

predict_btn = ttk.Button(button_frame, 
                        text="Predict Price", 
                        command=predict_price,
                        style='TButton')
predict_btn.pack(ipadx=20, ipady=5)

# Result display with card-like appearance
result_frame = ttk.Frame(main_container, style='TFrame')
result_frame.pack(pady=(0, 20))

# Card background with gradient
result_card = GradientFrame(result_frame, color1="#ffffff", color2="#f1f3f6",
                          width=300, height=100)
result_card.pack(padx=20, ipadx=20, ipady=10)

ttk.Label(result_card, 
          text="Estimated Value:", 
          font=('Helvetica', 12, 'bold'),
          background="#ffffff").place(relx=0.5, rely=0.3, anchor='center')

result_label = ttk.Label(result_card, 
                        text="₹0.00", 
                        font=('Helvetica', 24, 'bold'),
                        foreground="#2ecc71",
                        background="#ffffff")
result_label.place(relx=0.5, rely=0.7, anchor='center')

# Footer
footer = ttk.Frame(main_container, style='TFrame')
footer.pack(side='bottom', fill='x', pady=10)
ttk.Label(footer, 
          text="© 2023 House Price Prediction System | Professional Edition", 
          font=('Helvetica', 8), 
          foreground="#7f8c8d").pack()

root.mainloop()

c:\Users\sudee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\sudee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\sudee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\sudee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\sudee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py